# Replication Code for Experimenting with CNNs

The following contains the code I used to attempt a CNN from scratch on the dataset.

In [1]:
# Libraries
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os

# Seeds
np.random.seed(42)
tf.random.set_seed(42)

# Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

project_root_dir = 'your_directory_here'

In [3]:
aqi_dir = os.path.join(project_root_dir, 'aqi_data', 'final_data')
experiments_dir = os.path.join(project_root_dir, 'experiments')
img_dir = os.path.join(project_root_dir, 'img_data')

Basic custom model

Here, I employ a basic CNN design as in _Hands-On Machine Learning with Scikit-Learn, Keras, & Tensorflow_ (2 ed.) by Geron, pg. 461.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation = "relu", padding = "same", input_shape = [224, 224, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"),
    keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"),
    keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation = "softmax")
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])

In [ ]:
# Random horizontal and vertical flips for data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True
)

In [ ]:
train = np.load(os.path.join(experiments_dir, 'train.npz'))
valid = np.load(os.path.join(experiments_dir, 'valid.npz'))
test = np.load(os.path.join(experiments_dir, 'test.npz'))

In [ ]:
x_mean = train['x'].mean(axis = 0, keepdims = True)
x_std = train['x'].std(axis = 0, keepdims = True) + 1e-7
x_train = (train['x'] - x_mean) / x_std
x_valid = (valid['x'] - x_mean) / x_std

In [ ]:
from collections import Counter
item_ct = Counter(train['y'])
max_ct = float(max(item_ct.values()))
class_wts = {class_id: max_ct / num_img for class_id, num_img in item_ct.items()}

In [ ]:
batch_size = 16
epochs = 100

n_train = x_train.shape[0]

history = model.fit(datagen.flow(x_train, train['y'], batch_size = batch_size),
                    epochs = epochs, steps_per_epoch = n_train // batch_size,
                    validation_data = (x_valid, valid['y']),
                    class_weight = class_wts)

Training did not proceed well with the custom model given the small and noisy dataset. As in the best model, experiments with pre-trained networks proved more useful.